# Draguignan

In [82]:
# Activate multi-output in notebook
# %matplotlib inline (pour des graphes)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Parcours des données

### Chargement des données en mémoire

In [83]:
from os import getcwd
from pandas import DataFrame, read_csv


DATA_DIRECTORY = getcwd() + "/../data/"

# DGCL data
DATA_2020_PATH = DATA_DIRECTORY + "2020-communes-criteres-repartition.csv"
DATA_2021_PATH = DATA_DIRECTORY + "2021-communes-criteres-repartition.csv"

# Garanties DSU estimées
GARANTIES_DSU_PATH = DATA_DIRECTORY + "garanties_dsu.csv"

In [84]:
from scripts.load_dgcl_data import load_dgcl_file, adapt_dgcl_data

dgcl_data_2020 = load_dgcl_file(DATA_2020_PATH)
adapted_data_2020 = adapt_dgcl_data(dgcl_data_2020)

dgcl_data_2021 = load_dgcl_file(DATA_2021_PATH)
adapted_data_2021 = adapt_dgcl_data(dgcl_data_2021)


Loading /Users/sch/dev/gitruc/dotations-locales/data/2020-communes-criteres-repartition.csv


/Users/sch/.local/share/virtualenvs/dl/lib/python3.7/site-packages/ipykernel_launcher.py:3: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  This is separate from the ipykernel package so we can avoid doing imports until


Loading /Users/sch/dev/gitruc/dotations-locales/data/2021-communes-criteres-repartition.csv


/Users/sch/.local/share/virtualenvs/dl/lib/python3.7/site-packages/ipykernel_launcher.py:6: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  


In [85]:
# adapted_data_2021

## Draguignan

In [86]:
# Récupération des données d'une commune.

CODE_INSEE_DRAGUIGNAN = "83050"

# Données DGCL brutes (195 colonnes)

def get_dgcl_commune_data(code_insee_commune):
    return dgcl_data_2021.loc[dgcl_data_2021["Informations générales - Code INSEE de la commune"] == code_insee_commune]

draguignan_dgcl = get_dgcl_commune_data(CODE_INSEE_DRAGUIGNAN)
draguignan_dgcl.keys().tolist()  # toutes les colonnes

# Données adaptées avant intégration des communes nouvelles et des garanties DSU (34 colonnes)

def get_commune_data(data, code_insee_commune):
    return data.loc[data["Informations générales - Code INSEE de la commune"] == code_insee_commune]



# On récupère les données de la commune des jeux de données adaptés 
# car ils contiennent les mêmes noms de colonnes (pas toujours stables dans les données DGCL)
df_commune_2020 = get_commune_data(adapted_data_2020, CODE_INSEE_DRAGUIGNAN)
df_commune_2021 = get_commune_data(adapted_data_2021, CODE_INSEE_DRAGUIGNAN)

NOM_COMMUNE = df_commune_2021["Informations générales - Nom de la commune"].values[0]
print("Commune analysée : " + NOM_COMMUNE)
print("Données 2021")
# df_commune_2021

['Informations générales - Code INSEE de la commune',
 'Informations générales - Nom de la commune',
 'Informations générales - Code département de la commune',
 "Informations générales - Code SIREN de l'EPCI",
 "Informations générales - Nom de l'EPCI",
 'Informations générales - Exercice',
 'Informations générales - Régime fiscal EPCI',
 "Informations générales - Population DGF Année N'",
 'Informations générales - Population INSEE Année N ',
 "Informations générales - Résidences secondaires de l'année N",
 "Informations générales - Places de caravanes de l'année N avant majoration",
 "Informations générales - Coefficient de majoration des places de caravanes de l'année N",
 "Informations générales - Places de caravanes de l'année N après majoration",
 'Informations générales - Strate démographique Année N',
 'Informations générales - Superficie 2019',
 'Potentiel fiscal et financier des communes - Potentiel fiscal 3 taxes',
 'Potentiel fiscal et financier des communes - Potentiel fis

Commune analysée : DRAGUIGNAN
Données 2021


## Intégration de données complémentaires
Complétion des données par les évolutions de situation des communes

In [87]:
# Initialisation du nouveau jeu de données

fully_adapted_data_2021 = adapted_data_2021

### Ajout des garanties DSU

In [88]:
from pandas import read_csv
from scripts.load_dgcl_data import insert_dsu_garanties

# Chargement des garanties DSU toutes périodes
# TODO remplacer les garanties estimées pour 2021 par les garanties publiées par la DGCL

# TODO 2022 où toutes les valeurs sont à zéro + de nouvelles communes peuvent manquer
garanties_dsu = read_csv(GARANTIES_DSU_PATH, dtype={"Informations générales - Nom de la commune": str})

# Insertion des garanties 2021 au titre de la DSU (non calculées explicitement dans OFDL)
fully_adapted_data_2021 = insert_dsu_garanties(fully_adapted_data_2021, "2021", GARANTIES_DSU_PATH)

### Ajout des garanties communes nouvelles

In [89]:
from scripts.load_dgcl_data import insert_dsr_garanties_communes_nouvelles


# Insertion des garanties communes nouvelles au titre de la DSR (non calculées explicitement dans OFDL)
fully_adapted_data_2021 = insert_dsr_garanties_communes_nouvelles(fully_adapted_data_2021, "2021")

In [90]:
fully_adapted_data_2021.keys()  # 38 colonnes en 2021 (contre 34 initialement)

Index(['Informations générales - Code INSEE de la commune',
       'Informations générales - Nom de la commune', 'bureau_centralisateur',
       'chef_lieu_arrondissement', 'chef_lieu_de_canton',
       'chef_lieu_departement_dans_agglomeration', 'part_population_canton',
       'population_dgf', 'population_dgf_agglomeration',
       'population_dgf_departement_agglomeration', 'population_insee',
       'potentiel_financier', 'potentiel_financier_par_habitant',
       'revenu_total', 'strate_demographique', 'zrr', 'effort_fiscal',
       'longueur_voirie', 'zone_de_montagne', 'insulaire', 'superficie',
       'population_enfants', 'nombre_logements', 'nombre_logements_sociaux',
       'nombre_beneficiaires_aides_au_logement', 'population_qpv',
       'population_zfu', 'population_dgf_majoree',
       'recettes_reelles_fonctionnement', 'potentiel_fiscal',
       'population_dgf_maximum_commune_agglomeration', 'outre_mer',
       'population_dgf_chef_lieu_de_canton', 'revenu_par_habitan

### Calcul des dotations 2021


In [91]:
from openfisca_france_dotations_locales import CountryTaxBenefitSystem as DotationsLocales

from scripts.simulation import simulation_from_dgcl_csv
from scripts.load_dgcl_data import get_last_year_dotations


tbs = DotationsLocales()

results_2020_as_last_year = get_last_year_dotations(dgcl_data_2020)
data_2020_as_last_year = results_2020_as_last_year[[
    "Informations générales - Code INSEE de la commune", 
    "dsu_montant_eligible", 
    "dsr_montant_eligible_fraction_bourg_centre", 
    "dsr_montant_eligible_fraction_perequation", 
    "dsr_montant_hors_garanties_fraction_cible", 
    "population_dgf_majoree", 
    "dotation_forfaitaire"
]]



# Simulation à partir de données DGCL de la même année, complétées.
simulation_2021 = simulation_from_dgcl_csv(
    "2021", 
    fully_adapted_data_2021, 
    tbs, 
    data_previous_year=data_2020_as_last_year
)

In [92]:
# Quel est l'index de la commune nouvelle dans la simulation ?
import numpy as np

# for population in simulation_2021.populations.values():
#    if population.entity.is_person:
#        population.ids

# population_dgf = simulation_2021.calculate("population_dgf", 2021)
# population_dgf
# vimartin = np.where(population_dgf == 1303) > (array([ 3592, 11470, 12600, 19408, 21371, 26169, 26974]),)

recettes_reelles_fonctionnement = simulation_2021.calculate("recettes_reelles_fonctionnement", 2021)
indexes_draguignan = np.where(recettes_reelles_fonctionnement == 42871089.53)

indexes_draguignan # (array([32329]),)
index_draguignan = 32329

# Un autre indice pour confirmer que c'est bien Draguignan ?
revenu_par_habitant_moyen = simulation_2021.calculate("revenu_par_habitant_moyen", 2021)
revenu_par_habitant_moyen[index_draguignan] - 14792.537 < 0.001

(array([32329]),)

True

In [93]:
print("Dotations Draguignan : ")

print("--- DF")
dotation_forfaitaire = simulation_2021.calculate("dotation_forfaitaire", 2021)
print("dotation_forfaitaire : ", dotation_forfaitaire[index_draguignan])
print("df dgcl 2021 : ", draguignan_dgcl["Dotation forfaitaire - Dotation forfaitaire notifiée N"].values)

print("--- DSU")
dsu_montant = simulation_2021.calculate("dsu_montant", 2021)
print("dsu_montant : ", dsu_montant[index_draguignan])
print("dsu dgcl 2021 : ", draguignan_dgcl["Dotation de solidarité urbaine - Montant total réparti"].values)

print("--- DSR")
# dsr_garantie_commune_nouvelle_fraction_cible = simulation_2021.calculate("dsr_garantie_commune_nouvelle_fraction_cible", 2021)
# print("dsr_garantie_commune_nouvelle_fraction_cible : ", dsr_garantie_commune_nouvelle_fraction_cible[index_draguignan])
# dsr_garantie_commune_nouvelle_fraction_perequation = simulation_2021.calculate("dsr_garantie_commune_nouvelle_fraction_perequation", 2021)
# print("dsr_garantie_commune_nouvelle_fraction_perequation : ", dsr_garantie_commune_nouvelle_fraction_perequation[index_draguignan])
# dsr_garantie_commune_nouvelle_fraction_bourg_centre = simulation_2021.calculate("dsr_garantie_commune_nouvelle_fraction_bourg_centre", 2021)
# print("dsr_garantie_commune_nouvelle_fraction_bourg_centre : ", dsr_garantie_commune_nouvelle_fraction_bourg_centre[index_draguignan])

dotation_solidarite_rurale = simulation_2021.calculate("dotation_solidarite_rurale", 2021)
print("dotation_solidarite_rurale : ", dotation_solidarite_rurale[index_draguignan])

print("dsr dgcl 2021 - Fraction Cible : ", draguignan_dgcl["Dotation de solidarité rurale - Cible - Montant global réparti"].values)
print("dsr dgcl 2021 - Fraction Péréquation : ", draguignan_dgcl["Dotation de solidarité rurale - Péréquation - Montant global réparti (après garantie CN)"].values)
print("dsr dgcl 2021 - Fraction Bourg-Centre : ", draguignan_dgcl["Dotation de solidarité rurale Bourg-centre - Montant global réparti"].values)

Dotations Draguignan : 
--- DF
dotation_forfaitaire :  4287358.0
df dgcl 2021 :  [4291411]
--- DSU
dsu_montant :  1185469.0
dsu dgcl 2021 :  [1484416]
--- DSR
dotation_solidarite_rurale :  0.0
dsr dgcl 2021 - Fraction Cible :  [0]
dsr dgcl 2021 - Fraction Péréquation :  [0]
dsr dgcl 2021 - Fraction Bourg-Centre :  [0]


/Users/sch/dev/gitruc/openfisca-france-dotations-locales/openfisca_france_dotations_locales/variables/dotation_forfaitaire.py:188: RuntimeWarning: divide by zero encountered in log10
  dotation_supp_par_habitant = facteur_minimum + (facteur_maximum - facteur_minimum) * max_(0, min_(1, facteur_du_coefficient_logarithmique * log10(safe_divide(population_majoree_dgf, plancher_dgcl_population_dgf_majoree, 1))))
/Users/sch/dev/gitruc/openfisca-france-dotations-locales/openfisca_france_dotations_locales/variables/dotation_forfaitaire.py:40: RuntimeWarning: divide by zero encountered in log10
  coefficient_logarithmique = max_(1, min_(2, 1 + facteur_du_coefficient_logarithmique * log10(population_dgf / plancher_dgcl_population_dgf)))
/Users/sch/dev/gitruc/openfisca-france-dotations-locales/openfisca_france_dotations_locales/variables/dotation_forfaitaire.py:70: RuntimeWarning: divide by zero encountered in true_divide
  potentiel_fiscal_moyen_commune = where(population_logarithmee > 0, potent

### Réforme 2022

In [94]:
from scripts.load_dgcl_data import insert_dsu_garanties
from scripts.load_dgcl_data import insert_dsr_garanties_communes_nouvelles

fully_adapted_data_2022 = adapted_data_2021

fully_adapted_data_2022 = insert_dsu_garanties(fully_adapted_data_2022, "2022", GARANTIES_DSU_PATH)
fully_adapted_data_2022 = insert_dsr_garanties_communes_nouvelles(fully_adapted_data_2022, "2022")

In [95]:
from scripts.load_dgcl_data import get_last_year_dotations

results_2021_as_last_year = get_last_year_dotations(dgcl_data_2021)
data_2021_as_last_year = results_2021_as_last_year[
        [
            "Informations générales - Code INSEE de la commune",
            "dsu_montant_eligible",
            "dsr_montant_eligible_fraction_bourg_centre",
            "dsr_montant_eligible_fraction_perequation",
            "dsr_montant_hors_garanties_fraction_cible",
            "dotation_forfaitaire",
            "population_dgf_majoree",
        ]
    ]

In [96]:
from scripts.reform import DotationReform
from scripts.simulate_dotations import format_reforme_openfisca
from scripts.reformePLF_2022_dotations import plf_body_2022

tbs = DotationsLocales()

reforme_scenario = format_reforme_openfisca(plf_body_2022)
tbs_plf_2022 = DotationReform(tbs, reforme_scenario, 2022)

simulation_plf_2022 = simulation_from_dgcl_csv(2022, fully_adapted_data_2022, tbs_plf_2022, data_2021_as_last_year)

# recettes_reelles_fonctionnement = simulation_plf_2022.calculate("recettes_reelles_fonctionnement", 2022)
# print(np.where(recettes_reelles_fonctionnement == 42871089.53))
# index_draguignan = 32329

dsu_montant_2022 = simulation_plf_2022.calculate('dsu_montant', 2022)
dsu_montant_2022[index_draguignan]


(array([32329]),)


/Users/sch/dev/gitruc/openfisca-france-dotations-locales/openfisca_france_dotations_locales/variables/dotation_solidarite_urbaine.py:400: RuntimeWarning: invalid value encountered in true_divide
  facteur_qpv = (1 + np.where(population_insee > 0, poids_quartiers_prioritaires_ville * population_qpv / population_insee, 0))
/Users/sch/dev/gitruc/openfisca-france-dotations-locales/openfisca_france_dotations_locales/variables/dotation_solidarite_urbaine.py:401: RuntimeWarning: invalid value encountered in true_divide
  facteur_zfu = (1 + np.where(population_insee > 0, poids_zone_franche_urbaine * population_zfu / population_insee, 0))


945781.06

In [97]:
# !pip list